In [13]:
using Printf
using DataFrames
using CSV
using LinearAlgebra
# using DataStructures

include("RLalgo.jl")

### initialization of database

In [2]:
infile = "data/small.csv"
df = CSV.File(infile) |> DataFrame
# data_mat = Matrix(df);

x = [mod(df.s[j], 10)!=0 ? mod(df.s[j], 10) : 10 for j in 1:size(df,1)] 
y = [mod(df.s[j], 10)!=0 ? df.s[j] ÷ 10 + 1 : df.s[j] ÷ 10  for j in 1:size(df,1)] 

df = insertcols!(df, 2, :s_i => [x[i] for i in 1:size(df,1)])
df = insertcols!(df, 3, :s_j => [y[j] for j in 1:size(df,1)]);

xp = [mod(df.sp[j], 10)!=0 ? mod(df.sp[j], 10) : 10 for j in 1:size(df,1)] 
yp = [mod(df.sp[j], 10)!=0 ? df.sp[j] ÷ 10 + 1 : df.sp[j] ÷ 10  for j in 1:size(df,1)] 

df = insertcols!(df, 7, :sp_i => [xp[i] for i in 1:size(df,1)])
df = insertcols!(df, 8, :sp_j => [yp[j] for j in 1:size(df,1)])

50000×8 DataFrame
   Row │ s      s_i    s_j    a      r      sp     sp_i   sp_j  
       │ Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64 
───────┼────────────────────────────────────────────────────────
     1 │    85      5      9      3      0     86      6      9
     2 │    86      6      9      2      0     87      7      9
     3 │    87      7      9      3      0     97      7     10
     4 │    97      7     10      2      0     87      7      9
     5 │    87      7      9      1      0     86      6      9
     6 │    86      6      9      3      0     76      6      8
     7 │    76      6      8      4      0     66      6      7
     8 │    66      6      7      1      0     65      5      7
     9 │    65      5      7      2      0     66      6      7
    10 │    66      6      7      3      0     76      6      8
    11 │    76      6      8      2      0     66      6      7
   ⋮   │   ⋮      ⋮      ⋮      ⋮      ⋮      ⋮      ⋮      ⋮
 49991 │    89      9      9      1      0     79      9      8
 49992 │    79      9      8      2      0     80     10      8
 49993 │    80     10      8      3      0     79      9      8
 49994 │    79      9      8      1      0     78      8      8
 49995 │    78      8      8      3      0     77      7      8
 49996 │    77      7      8      2      0     78      8      8
 49997 │    78      8      8      1      0     77      7      8
 49998 │    77      7      8      4      0     67      7      7
 49999 │    67      7      7      1      0     66      6      7
 50000 │    66      6      7      3      0     76      6      8
                                              49979 rows omitted

In [3]:
idx = findall(df.r .> 0)
k = rand([2,19,5,6,10])

5

### Gradient Q-learning (not deep, w/o experience replay)

In [4]:
S = [[x,y] for x in 1:10, y in 1:10]   # FIXME: need to reshape this 
# S = [i for i in 1:100]
A = [1,2,3,4]
γ = 0.95
T = NaN
R = NaN
TR = NaN

prob = MDP(γ,S,A,T,R,TR)

MDP(0.95, [[1, 1] [1, 2] … [1, 9] [1, 10]; [2, 1] [2, 2] … [2, 9] [2, 10]; … ; [9, 1] [9, 2] … [9, 9] [9, 10]; [10, 1] [10, 2] … [10, 9] [10, 10]], [1, 2, 3, 4], NaN, NaN, NaN)

In [17]:
β(s,a) = [s[1],s[2], s[1]+s[2], s[1]^2, s[2]^2, s[1]*s[2], a, a^2, 1]   # basis funciton 
Q(θ,s,a) = dot(θ,β(s,a)) 
∇Q(θ,s,a) = β(s,a) 
θ = [0.1 for i in 1:9] # initial parameter vector

α = 0.5 # learning rate 
model = GradientQLearning(prob.A, prob.γ, Q, ∇Q, θ, α)

GradientQLearning([1, 2, 3, 4], 0.95, Q, ∇Q, [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 0.5)

In [18]:
train_offline(prob, model, df, 10000) 

[-0.04832, -0.0302, -0.07852, -0.3866, -0.151, -0.2416, -0.02416, -0.09665, -0.00604]
[0.05214, 0.02607, 0.07821, 0.4171, 0.1043, 0.2086, 0.02607, 0.1043, 0.006518]
[-0.03055, -0.04888, -0.07943, -0.1527, -0.391, -0.2444, -0.01833, -0.05499, -0.00611]
[0.01161, 0.05223, 0.06383, 0.02321, 0.47, 0.1045, 0.02321, 0.09284, 0.005803]
[-0.0435, -0.0522, -0.09571, -0.2175, -0.3132, -0.261, -0.0348, -0.1392, -0.008701]
[0.05711, 0.03264, 0.08975, 0.3998, 0.1305, 0.2285, 0.02448, 0.07343, 0.008159]
[0.02137, 0.04809, 0.06946, 0.08549, 0.4328, 0.1924, 0.02137, 0.08549, 0.005343]
[0.057, 0.03257, 0.08957, 0.399, 0.1303, 0.228, 0.02443, 0.07329, 0.008143]
[0.05749, 0.03285, 0.09034, 0.4024, 0.1314, 0.2299, 0.01642, 0.03285, 0.008212]
[0.03669, 0.05137, 0.08806, 0.1834, 0.3596, 0.2568, 0.02201, 0.06604, 0.007338]
[0.01167, 0.05251, 0.06418, 0.02334, 0.4726, 0.105, 0.0175, 0.05251, 0.005835]
[0.03413, 0.08531, 0.1194, 0.06825, 0.4266, 0.1706, 0.03413, 0.06825, 0.01706]
[-0.06104, -0.09156, -0.1526, 

[0.05147, 0.02941, 0.08088, 0.3603, 0.1176, 0.2059, 0.007353, 0.007353, 0.007353]
[0.07665, 0.07665, 0.1533, 0.23, 0.23, 0.23, 0.02555, 0.02555, 0.02555]
[-0.06201, -0.031, -0.09301, -0.3721, -0.09301, -0.186, -0.01033, -0.01033, -0.01033]
[-0.05783, -0.02891, -0.08674, -0.347, -0.08674, -0.1735, -0.03855, -0.1542, -0.009638]
[-0.01047, -0.04709, -0.05756, -0.02093, -0.4238, -0.09419, -0.005233, -0.005233, -0.005233]
[0.04941, 0.02823, 0.07764, 0.3458, 0.1129, 0.1976, 0.02823, 0.1129, 0.007058]
[-0.06683, -0.02228, -0.0891, -0.401, -0.04455, -0.1337, -0.01114, -0.01114, -0.01114]
[0.06683, 0.02228, 0.0891, 0.2005, 0.02228, 0.06683, 0.0891, 0.3564, 0.02228]
[-0.04604, -0.01023, -0.05627, -0.4144, -0.02046, -0.09208, -0.02046, -0.08185, -0.005116]
[0.06015, 0.03007, 0.09022, 0.3609, 0.09022, 0.1804, 0.03007, 0.09022, 0.01002]
[0.06767, 0.0406, 0.1083, 0.3384, 0.1218, 0.203, 0.02707, 0.05414, 0.01353]
[-0.04042, -0.02246, -0.06288, -0.3638, -0.1123, -0.2021, -0.004491, -0.004491, -0.00449

[-0.07086, -0.03543, -0.1063, -0.2835, -0.07086, -0.1417, -0.05315, -0.1594, -0.01772]
[-0.05111, -0.007301, -0.05841, -0.3578, -0.007301, -0.05111, -0.02921, -0.1168, -0.007301]
[0.02461, 0.02461, 0.04922, 0.2215, 0.2215, 0.2215, 0.005469, 0.01094, 0.002735]
[-0.009151, -0.04118, -0.05033, -0.0183, -0.3706, -0.08236, -0.01373, -0.04118, -0.004575]
[0.04494, 0.02568, 0.07063, 0.3146, 0.1027, 0.1798, 0.01284, 0.02568, 0.006421]
[0.04501, 0.02572, 0.07073, 0.3151, 0.1029, 0.18, 0.00643, 0.00643, 0.00643]
[0.04485, 0.02563, 0.07049, 0.314, 0.1025, 0.1794, 0.01282, 0.02563, 0.006408]
[0.04439, 0.02536, 0.06975, 0.3107, 0.1015, 0.1775, 0.01902, 0.05707, 0.006341]
[-0.02432, -0.02189, -0.04621, -0.2432, -0.197, -0.2189, -0.004864, -0.009728, -0.002432]
[0.009096, 0.04093, 0.05003, 0.01819, 0.3684, 0.08186, 0.01364, 0.04093, 0.004548]
[0.03004, 0.02336, 0.0534, 0.2704, 0.1635, 0.2103, 0.006675, 0.01335, 0.003338]
[-0.04316, -0.02467, -0.06783, -0.3022, -0.09866, -0.1727, -0.02467, -0.09866, -

[0.008182, 0.03682, 0.045, 0.01636, 0.3314, 0.07364, 0.004091, 0.004091, 0.004091]
[-0.008174, -0.03678, -0.04496, -0.01635, -0.331, -0.07356, -0.004087, -0.004087, -0.004087]
[-0.03859, -0.02205, -0.06064, -0.2701, -0.0882, -0.1544, -0.02205, -0.0882, -0.005513]
[-0.005239, -0.04191, -0.04715, -0.005239, -0.3353, -0.04191, -0.005239, -0.005239, -0.005239]
[-0.05574, -0.1115, -0.1672, -0.05574, -0.2229, -0.1115, -0.05574, -0.05574, -0.05574]
[-0.0394, -0.02252, -0.06192, -0.2758, -0.09007, -0.1576, -0.01689, -0.05066, -0.005629]
[-0.03984, -0.02276, -0.0626, -0.2789, -0.09106, -0.1594, -0.005691, -0.005691, -0.005691]
[0.008115, 0.03652, 0.04463, 0.01623, 0.3286, 0.07303, 0.008115, 0.01623, 0.004057]
[-0.03311, -0.01472, -0.04783, -0.298, -0.05886, -0.1324, -0.01472, -0.05886, -0.003679]
[-0.03972, -0.0227, -0.06242, -0.278, -0.09079, -0.1589, -0.005674, -0.005674, -0.005674]
[0.03958, 0.02262, 0.0622, 0.2771, 0.09047, 0.1583, 0.01131, 0.02262, 0.005654]
[0.006607, 0.03303, 0.03964, 0.

[-0.02821, -0.02418, -0.05239, -0.1975, -0.1451, -0.1692, -0.00403, -0.00403, -0.00403]
[-0.007176, -0.03229, -0.03947, -0.01435, -0.2906, -0.06458, -0.01076, -0.03229, -0.003588]
[0.01309, 0.02618, 0.03927, 0.06544, 0.2618, 0.1309, 0.005235, 0.01047, 0.002618]
[-0.007073, -0.03183, -0.0389, -0.01415, -0.2864, -0.06365, -0.01415, -0.05658, -0.003536]
[0.03517, 0.0201, 0.05527, 0.2462, 0.0804, 0.1407, 0.01005, 0.0201, 0.005025]
[0.007059, 0.03176, 0.03882, 0.01412, 0.2859, 0.06353, 0.01412, 0.05647, 0.003529]
[0.02021, 0.02695, 0.04717, 0.1213, 0.2156, 0.1617, 0.01011, 0.03032, 0.003369]
[0.007133, 0.0321, 0.03923, 0.01427, 0.2889, 0.0642, 0.0107, 0.0321, 0.003566]
[0.03503, 0.02002, 0.05505, 0.2452, 0.08008, 0.1401, 0.01001, 0.02002, 0.005005]
[-0.01724, -0.01724, -0.03447, -0.1724, -0.1724, -0.1724, -0.005171, -0.01551, -0.001724]
[0.007763, 0.04658, 0.05434, 0.007763, 0.2795, 0.04658, 0.02329, 0.06987, 0.007763]
[-0.03378, -0.0193, -0.05308, -0.2365, -0.07721, -0.1351, -0.0193, -0.07

...

[0.0003062, 0.000175, 0.0004811, 0.002143, 0.0006998, 0.001225, 4.374e-5, 4.374e-5, 4.374e-5]
[-6.236e-5, -0.0002806, -0.000343, -0.0001247, -0.002526, -0.0005613, -6.236e-5, -0.0001247, -3.118e-5]
[0.0003048, 0.0001742, 0.000479, 0.002134, 0.0006967, 0.001219, 8.709e-5, 0.0001742, 4.354e-5]
[6.224e-5, 0.0002801, 0.0003423, 0.0001245, 0.002521, 0.0005602, 6.224e-5, 0.0001245, 3.112e-5]
[0.0003013, 0.0001722, 0.0004735, 0.002109, 0.0006887, 0.001205, 0.0001291, 0.0003874, 4.305e-5]
[-0.0001128, -0.0002537, -0.0003665, -0.000451, -0.002283, -0.001015, -0.0001128, -0.000451, -2.819e-5]
[-0.0002255, -0.0001127, -0.0003382, -0.002255, -0.0005637, -0.001127, -4.51e-5, -9.019e-5, -2.255e-5]
[-0.0002933, -0.0001676, -0.0004609, -0.002053, -0.0006705, -0.001173, -0.0001676, -0.0006705, -4.19e-5]
[-0.0003001, -0.0001715, -0.0004716, -0.002101, -0.000686, -0.0012, -0.0001286, -0.0003859, -4.287e-5]
[6.195e-5, 0.0002788, 0.0003407, 0.0001239, 0.002509, 0.0005575, 3.097e-5, 3.097e-5, 3.097e-5]
[0.0

[-0.0002667, -0.0001524, -0.0004191, -0.001867, -0.0006097, -0.001067, -0.0001524, -0.0006097, -3.81e-5]
[-5.639e-5, -0.0002537, -0.0003101, -0.0001128, -0.002284, -0.0005075, -2.819e-5, -2.819e-5, -2.819e-5]
[0.000282, 0.000188, 0.00047, 0.001692, 0.000752, 0.001128, 0.000188, 0.000752, 4.7e-5]
[-0.0002756, -0.0001575, -0.0004332, -0.001929, -0.00063, -0.001103, -3.938e-5, -3.938e-5, -3.938e-5]
[0.0002657, 0.0001518, 0.0004175, 0.00186, 0.0006072, 0.001063, 0.0001518, 0.0006072, 3.795e-5]
[0.0002154, 0.0001197, 0.0003351, 0.001939, 0.0005984, 0.001077, 9.574e-5, 0.000383, 2.393e-5]
[-0.0001552, -0.0002715, -0.0004267, -0.0006207, -0.001901, -0.001086, -0.0001164, -0.0003491, -3.879e-5]
[0.0002189, 0.0002189, 0.0004378, 0.001313, 0.001313, 0.001313, 0.0001095, 0.0003284, 3.648e-5]
[5.565e-5, 0.0002504, 0.0003061, 0.0001113, 0.002254, 0.0005008, 8.347e-5, 0.0002504, 2.782e-5]
[0.0002707, 0.0001547, 0.0004254, 0.001895, 0.0006188, 0.001083, 0.000116, 0.0003481, 3.868e-5]
[0.0002705, 0.00

[8.25e-5, 0.0002888, 0.0003713, 0.000165, 0.002021, 0.0005775, 8.25e-5, 0.000165, 4.125e-5]
[-0.000243, -0.0001389, -0.0003819, -0.001701, -0.0005555, -0.0009721, -0.0001389, -0.0005555, -3.472e-5]
[-0.0002133, -0.0001333, -0.0003466, -0.001706, -0.0006665, -0.001066, -5.332e-5, -0.0001066, -2.666e-5]
[-8.985e-5, -0.0002396, -0.0003294, -0.0002695, -0.001917, -0.0007188, -0.0001198, -0.0004792, -2.995e-5]
[0.0002482, 0.0001418, 0.00039, 0.001737, 0.0005672, 0.0009926, 0.0001064, 0.0003191, 3.545e-5]
[0.0002714, 0.0001163, 0.0003877, 0.0019, 0.000349, 0.0008142, 7.755e-5, 0.0001551, 3.877e-5]
[0.00025, 0.0001429, 0.0003929, 0.00175, 0.0005715, 0.001, 7.144e-5, 0.0001429, 3.572e-5]
[5.018e-5, 0.0002258, 0.000276, 0.0001004, 0.002032, 0.0004516, 0.0001004, 0.0004014, 2.509e-5]
[-5.107e-5, -0.0002298, -0.0002809, -0.0001021, -0.002068, -0.0004596, -2.553e-5, -2.553e-5, -2.553e-5]
[-0.0002411, -0.0001378, -0.0003789, -0.001688, -0.0005511, -0.0009644, -0.0001378, -0.0005511, -3.444e-5]
[-0.

Excessive output truncated after 524297 bytes.

-0.0005199, -0.0009099, -3.25e-5, -3.25e-5, -3.25e-5]
[0.0002272, 0.0001299, 0.0003571, 0.001591, 0.0005194, 0.000909, 3.246e-5, 3.246e-5, 3.246e-5]
[0.0004052, 0.0004052, 0.0008104, 0.0008104, 0.0008104, 0.0008104, 0.0004052, 0.0008104, 0.0002026]
[-0.000178, -0.0002136, -0.0003916, -0.0008901, -0.001282, -0.001068, -3.56e-5, -3.56e-5, -3.56e-5]
[-0.0002266, -0.0001295, -0.000356, -0.001586, -0.0005178, -0.0009062, -3.237e-5, -3.237e-5, -3.237e-5]
[-0.0002184, -0.0001248, -0.0003431, -0.001529, -0.0004991, -0.0008734, -0.0001248, -0.0004991, -3.119e-5]
[4.588e-5, 0.0002064, 0.0002523, 9.175e-5, 0.001858, 0.0004129, 6.881e-5, 0.0002064, 2.294e-5]
[-0.0001174, -0.0001879, -0.0003053, -0.0005872, -0.001503, -0.0009395, -9.395e-5, -0.0003758, -2.349e-5]
[0.000102, 0.0001457, 0.0002478, 0.0007141, 0.001457, 0.00102, 2.915e-5, 5.83e-5, 1.457e-5]
[0.0002439, 0.0001045, 0.0003484, 0.001707, 0.0003135, 0.0007316, 6.967e-5, 0.0001393, 3.484e-5]
[8.193e-5, 0.0002185, 0.0003004, 0.0002458, 0.0017

In [19]:
a_opt = [findmax([model.Q(model.θ, s, a) for a in A])[2] for s in [[x,y] for x in 1:10, y in 1:10]]

10×10 Matrix{Int64}:
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4
 4  4  4  4  4  4  4  4  4  4

In [20]:
Q_opt = [[model.Q(model.θ, s, a) for a in A] for s in [[x,y] for x in 1:10, y in 1:10]]

10×10 Matrix{Vector{Float64}}:
 [10.3776, 11.1889, 11.755, 12.0761]   …  [79.3099, 80.1211, 80.6873, 81.0084]
 [16.7612, 17.5724, 18.1386, 18.4597]     [86.9593, 87.7705, 88.3367, 88.6578]
 [24.8669, 25.6781, 26.2443, 26.5654]     [96.3307, 97.1419, 97.7081, 98.0292]
 [34.6946, 35.5059, 36.072, 36.3931]      [107.424, 108.235, 108.802, 109.123]
 [46.2445, 47.0557, 47.6219, 47.943]      [120.24, 121.051, 121.617, 121.938]
 [59.5164, 60.3277, 60.8939, 61.215]   …  [134.778, 135.589, 136.155, 136.476]
 [74.5105, 75.3217, 75.8879, 76.209]      [151.037, 151.849, 152.415, 152.736]
 [91.2267, 92.0379, 92.6041, 92.9252]     [169.019, 169.831, 170.397, 170.718]
 [109.665, 110.476, 111.042, 111.363]     [188.723, 189.535, 190.101, 190.422]
 [129.825, 130.637, 131.203, 131.524]     [210.15, 210.961, 211.527, 211.848]

In [21]:
model.θ

9-element Vector{Float64}:
  1.2073741528997048
  1.345028359407278
  2.452402512306988
  0.8610491829590974
  0.33827904574790435
  0.14064233779597782
  1.178825135681523
 -0.12253039072951107
  0.5241562222420969

In [ ]:
ε = 0.1 # probability of random action 
π = EpsilonGreedyExploration(ε) 
k = 20 # number of steps to simulate 
s = 0.0 # initial state 
simulate(𝒫, model, π, k, s)

In [ ]:

capacity = 100 # maximum size of the replay buffer 
ExperienceTuple = Tuple{Vector,Float64,Float64,Vector} 
M = CircularBuffer{ExperienceTuple}(capacity) # replay buffer 
m_grad = 20 # batch size 
model = ReplayGradientQLearning(𝒫.𝒜, 𝒫.γ, Q, ∇Q, θ, α, M, m, m_grad)
